俳優を4グループ分割するための年齢閾値発見のため

In [ ]:
import pandas as pd
import numpy as np

# CSVファイルを読み込み
df1 = pd.read_csv('../data/CREMA-D/raw/VideoDemographics.csv')
df1_target = df1.copy()
df2 = pd.read_csv('../data/CREMA-D/raw/SentenceFilenames.csv')
df2_target = df2.copy()

# 実際の列名を使用
actor_id_column = 'ActorID'
age_column = 'Age'
gender_column = 'Sex'

# 各役者がgroupのどこに属するかを確認
actor_dct = {}

def create_dct(df_target, actor_id_col, age_col, gender_col):
    age_threshold = 40  # 閾値
    
    for _, row in df_target.iterrows():
        actor_id = row[actor_id_col]  # 変数名修正
        age = row[age_col]
        gender = row[gender_col]
            
        if age <= age_threshold:
            if gender == 'Male':  
                actor_dct[actor_id] = 1
            else:  
                actor_dct[actor_id] = 3
        else: 
            if gender == 'Male':
                actor_dct[actor_id] = 2
            else:
                actor_dct[actor_id] = 4

def count_groups(df_target, actor_dct):
    groups = {1: 0, 2: 0, 3: 0, 4: 0}
    for _, row in df_target.iterrows():
        filename = row["Filename"]
        
        # ファイル名から ActorID を抽出（数値に変換）
        try:
            actor_id = int(filename.split('_')[0])
            
            # actor_dctにキーが存在するか確認
            if actor_id in actor_dct:
                group = actor_dct[actor_id]
                groups[group] += 1
            else:
                print(f"Actor ID {actor_id} not found in demographics data")
                
        except (ValueError, IndexError) as e:
            print(f"Error processing filename '{filename}': {e}")
            continue
            
    return groups

# グループ分割を実行
create_dct(df1_target, actor_id_column, age_column, gender_column)
groups = count_groups(df2_target, actor_dct)

print("\n=== グループ分割結果 ===")
print("Group 1 (若 男性):", groups[1], "件")
print("Group 2 (古 男性):", groups[2], "件") 
print("Group 3 (若 女性):", groups[3], "件")
print("Group 4 (古 女性):", groups[4], "件")

# 統計情報の表示
print("\n=== 詳細統計 ===")
print(f"総俳優数: {len(actor_dct)}人")
print(f"総ファイル数: {len(df2_target)}件")

# 性別分布の確認
male_count = len(df1_target[df1_target['Sex'] == 'Male'])
female_count = len(df1_target[df1_target['Sex'] == 'Female'])
print(f"\n=== 性別分布 ===")
print(f"男性: {male_count}人")
print(f"女性: {female_count}人")


=== グループ分割結果 ===
Group 1 (若 男性): 2372 件
Group 2 (古 男性): 1558 件
Group 3 (若 女性): 2452 件
Group 4 (古 女性): 1060 件

=== 詳細統計 ===
総俳優数: 91人
総ファイル数: 7442件
Group 1: 29人, 1人あたり平均81.8ファイル
Group 2: 19人, 1人あたり平均82.0ファイル
Group 3: 30人, 1人あたり平均81.7ファイル
Group 4: 13人, 1人あたり平均81.5ファイル

=== 性別分布 ===
男性: 48人
女性: 43人


各 テキスト×感情×group ペア　のXXだけサンプルデータ数 

In [6]:
import pandas as pd

# CSVファイルを読み込み
df1 = pd.read_csv('../data/CREMA-D/raw/VideoDemographics.csv')
df1_target = df1.copy()
df2 = pd.read_csv('../data/CREMA-D/raw/SentenceFilenames.csv')
df2_target = df2.copy()

# 実際の列名を使用
actor_id_column = 'ActorID'
age_column = 'Age'
gender_column = 'Sex'

# 各役者がgroupのどこに属するかを確認
actor_dct = {}

def create_actor_dct(df_target, actor_id_col, age_col, gender_col):
    age_threshold = 40  # 閾値
    
    for _, row in df_target.iterrows():
        actor_id = row[actor_id_col]
        age = row[age_col]
        gender = row[gender_col]
            
        if age <= age_threshold:
            if gender == 'Male':  
                actor_dct[actor_id] = 1
            else:  
                actor_dct[actor_id] = 3
        else: 
            if gender == 'Male':
                actor_dct[actor_id] = 2
            else:
                actor_dct[actor_id] = 4


def count_groups(df_target, actor_dct):
    dct = {}  # テキスト_感情_group -> data数
    # 重複 = 0

    for _, row in df_target.iterrows():
        filename = row["Filename"]
        part = filename.split('_')
        
        # ファイル名から ActorID を抽出（数値に変換）
        actor_id = int(part[0])
        sentence_emotion = part[1] + "_" + part[2]
        strong = part[3]

        # "XX"で終わらないファイルは無視
        if not strong == "XX":
            continue

        actor_group = actor_dct[actor_id]
        sentence_emotion_group = sentence_emotion + f"_{actor_group}"

        if sentence_emotion_group not in dct:
            dct[sentence_emotion_group] = 0
        
        # 既存のリストを取得
        dct[sentence_emotion_group] += 1

    return dct


# グループ分割を実行
create_actor_dct(df1_target, actor_id_column, age_column, gender_column)
dct = count_groups(df2_target, actor_dct)

print("=== テキスト_感情_group別 data数 ===")
sum = 0
for text_emotion in sorted(dct.keys()):
    print(f"{text_emotion}: {dct[text_emotion]}人")
    sum += dct[text_emotion]

print(f"\n総データ数: {sum}")
print(f"\n総パターン数: {len(dct)}")

=== テキスト_感情_group別 data数 ===
DFA_ANG_1: 29人
DFA_ANG_2: 19人
DFA_ANG_3: 30人
DFA_ANG_4: 13人
DFA_DIS_1: 29人
DFA_DIS_2: 19人
DFA_DIS_3: 30人
DFA_DIS_4: 13人
DFA_FEA_1: 29人
DFA_FEA_2: 19人
DFA_FEA_3: 30人
DFA_FEA_4: 13人
DFA_HAP_1: 29人
DFA_HAP_2: 19人
DFA_HAP_3: 30人
DFA_HAP_4: 13人
DFA_NEU_1: 29人
DFA_NEU_2: 19人
DFA_NEU_3: 30人
DFA_NEU_4: 13人
DFA_SAD_1: 29人
DFA_SAD_2: 19人
DFA_SAD_3: 30人
DFA_SAD_4: 13人
IEO_NEU_1: 29人
IEO_NEU_2: 19人
IEO_NEU_3: 30人
IEO_NEU_4: 13人
IOM_ANG_1: 29人
IOM_ANG_2: 19人
IOM_ANG_3: 30人
IOM_ANG_4: 13人
IOM_DIS_1: 29人
IOM_DIS_2: 19人
IOM_DIS_3: 30人
IOM_DIS_4: 13人
IOM_FEA_1: 29人
IOM_FEA_2: 19人
IOM_FEA_3: 30人
IOM_FEA_4: 13人
IOM_HAP_1: 29人
IOM_HAP_2: 19人
IOM_HAP_3: 30人
IOM_HAP_4: 13人
IOM_NEU_1: 29人
IOM_NEU_2: 19人
IOM_NEU_3: 30人
IOM_NEU_4: 13人
IOM_SAD_1: 29人
IOM_SAD_2: 19人
IOM_SAD_3: 30人
IOM_SAD_4: 13人
ITH_ANG_1: 28人
ITH_ANG_2: 19人
ITH_ANG_3: 30人
ITH_ANG_4: 13人
ITH_DIS_1: 28人
ITH_DIS_2: 19人
ITH_DIS_3: 30人
ITH_DIS_4: 13人
ITH_FEA_1: 28人
ITH_FEA_2: 19人
ITH_FEA_3: 30人
ITH_FEA_4: 13人
ITH_HAP_1: 

XXの数だけを数える

In [7]:
import pandas as pd

# CSVファイルを読み込み
df = pd.read_csv('../data/CREMA-D/raw/SentenceFilenames.csv')

# Filenameが"XX"で終わるものをフィルタリング
xx_endings = df[df['Filename'].str.endswith('XX')]

# 件数をカウント
xx_count = len(xx_endings)

print(f"Filenameが'XX'で終わるもの: {xx_count}件")
print(f"全体のファイル数: {len(df)}件")
print(f"割合: {xx_count/len(df)*100:.1f}%")

Filenameが'XX'で終わるもの: 6077件
全体のファイル数: 7442件
割合: 81.7%
